In [1]:
# This notebook shows how to get the beginning and end boundaries of a time concept, 
# and the duration betweent the two boundaries. 

# We use Jurassic as an example

# We will fist find the scheme ID for the latest version of the global geological time scale

install.packages("SPARQL")
library(SPARQL)

Installing package into ‘/nethome/max/.R-libs’
(as ‘lib’ is unspecified)

Loading required package: XML

Loading required package: RCurl



In [6]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"
  
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = " 
                prefix dc: <http://purl.org/dc/elements/1.1/>
                prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#>
                prefix skos: <http://www.w3.org/2004/02/skos/core#>
                prefix time: <http://www.w3.org/2006/time#>
                prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
                prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
        "

In [8]:
# the query code for find the latest global vocabulary scheme 
 
q = paste0(
              sparql_prefix, 
                  '
                   SELECT DISTINCT ?schURI ?lbl
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                        ?schURI a skos:ConceptScheme ;
                                rdfs:label ?lbl .
                        FILTER regex(?lbl, "International", "i")
                     }      
                   }
                   ORDER BY DESC (?schURI)
                   LIMIT 1
                  '
              )

# run the query  
resScheme = SPARQL(endpoint, q)$results
  

In [9]:
# the query code
 
q = paste0(
              sparql_prefix, 
                  '
                   SELECT   DISTINCT ?begTimeValue ?endTimeValue
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                          { #for the pattern of local vocabulary 
                                 ?tconcept  a gts:GeochronologicEra ;  
                                                 rdfs:label ?label .
                                 FILTER (lang(?label) = "en")
                                 FILTER strstarts(?label, "', 'Jurassic', '").
                                 ?tconcept time:hasBeginning ?beg ;
                                 time:hasEnd ?end .
                                 ?beg time:inTemporalPosition ?begTime .
                                 ?end time:inTemporalPosition ?endTime .
                                 ?begTime time:numericPosition ?begTimeValue .
                                 ?endTime time:numericPosition ?endTimeValue .
                          }
                       UNION
                          { #for the pattern of local vocabulary 
                             ?tconcept a gts:GeochronologicEra ;  
                                       rdfs:label ?label .
                             FILTER (lang(?label) = "en")
                             FILTER strstarts(?label, "', 'Jurassic', '").
                             ?tconcept dc:description
                             [
                                time:hasBeginning ?beg ;
                                time:hasEnd ?end ;
                                skos:inScheme  ', resScheme$schURI[1], '
                             ].
                             
                          ?beg time:inTemporalPosition ?begTime .
                          ?end time:inTemporalPosition ?endTime .
                          
                          ?begTime dc:description
                              [
                                 time:numericPosition ?begTimeValue ;
                                 skos:inScheme  ', resScheme$schURI[1], '
                               ].
                          ?endTime dc:description
                              [
                                 time:numericPosition ?endTimeValue ;
                                 skos:inScheme  ', resScheme$schURI[1], '
                              ] .
                             }
                     }
                   }
                  '
              )

In [10]:
# run the query  
res = SPARQL(endpoint, q)$results
  
# calculate the druation
res$duration = abs(res[[1]]-res[[2]])
  
return(res)

begTimeValue endTimeValue duration
1 201.3        145          56.3